In [2]:
import gzip
import numpy
import random
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)
### Rating baseline: compute averages for each user, or return the global average if we've never seen the user before
allRatings = []
userRatings = defaultdict(list)
data = list(readGz("Assignment1/HW3/assignment1/train.json.gz"))

In [4]:
# initialzation
users_perItem = defaultdict(list)
items_perUser = defaultdict(list)
userItem_rating = {} # rating for a specific user with respect to a specific item
user_bias = {}
item_bias = {}
alpha = 0
lam = 6

prev_MSE = 20 #randomly initializing previous MSE
MSE = 10 #randomly intitializing current MSE

# store the parameters that result in smallest MSE
user_bias_final = {}
item_bias_final = {}
alpha_final = 0

for l in data:#[:100000]:
    user,item = l['reviewerID'],l['itemID']
    users_perItem[item].append(l['reviewerID'])
    items_perUser[user].append(l['itemID'])
    userItem_rating[user+item] = l['rating']
    user_bias[user] = 0
    item_bias[item] = 0

In [5]:
prev_MSE = 10 #randomly initializing previous MSE
MSE = 9 #randomly intitializing current MSE
while MSE < prev_MSE:#abs(prev_MSE - MSE) > 0.00001:
    prev_MSE = MSE
    ## update alpha
    alpha = [l['rating'] - (user_bias[l['reviewerID']] + item_bias[l['itemID']]) for l in data]#[:100000]]
    alpha = sum(alpha) / 200000#100000
    
    ## update user bias
    for u in user_bias:
        temp = sum([userItem_rating[u+i] - (alpha + item_bias[i]) for i in items_perUser[u]])
        user_bias[u] = temp / (lam + len(items_perUser[u]))
    
    ## update item bias
    for i in item_bias:
        temp = sum([userItem_rating[u+i] - (alpha + user_bias[u]) for u in users_perItem[i]])
        item_bias[i] = temp / (lam + len(users_perItem[i]))
        
    ## compute MSE
    diff = [(l['rating'] - (alpha + user_bias[l['reviewerID']] + item_bias[l['itemID']]))**2 for l in data]#[:100000]]
    MSE = sum(diff) / len(diff)

    
    if MSE < prev_MSE:
        ## update final results
        for u in user_bias:
            user_bias_final[u] = user_bias[u]
        for i in item_bias:
            item_bias_final[i] = item_bias[i]
        alpha_final = alpha

In [6]:
## Validation MSE
diff = []
for l in data[100000:200000]:
    if l['reviewerID'] not in user_bias and l['itemID'] not in item_bias:
        diff.append((l['rating'] - alpha)**2)
    elif l['reviewerID'] not in user_bias:
        diff.append((l['rating'] - (alpha + item_bias[l['itemID']]))**2)
    elif l['itemID'] not in item_bias:
        diff.append((l['rating'] - (alpha + user_bias[l['reviewerID']]))**2)
    else:
        diff.append((l['rating'] - (alpha + user_bias[l['reviewerID']] + item_bias[l['itemID']]))**2)
MSE = sum(diff) / len(diff)

print 'MSE = ', MSE,'for validating set (lambda = 6)'

MSE =  0.816191926778 for validating set (lambda = 6)


In [7]:
userItemRating = []

predictions = open("Assignment1/HW3/assignment1/predictions_Rating.txt", 'w')
for l in open("Assignment1/HW3/assignment1/pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u, i = l.strip().split('-')
    if u not in user_bias_final and i not in item_bias_final:
        predicted_rating = alpha_final
    elif u not in user_bias_final:
        predicted_rating = alpha_final + item_bias_final[i]
    elif i not in item_bias_final:
        predicted_rating = alpha_final + user_bias_final[u]
    else:
        predicted_rating = alpha_final + user_bias_final[u] + item_bias_final[i]# + gamma_inner
        
    if predicted_rating > 5.0:
        predictions.write(u + '-' + i + ',' + str(5.0) + '\n')
    else:
        predictions.write(u + '-' + i + ',' + str(predicted_rating) + '\n')
            
predictions.close()